In [1]:
PYTHON_PATHS = [".."]
import sys
for path in PYTHON_PATHS:
    if path not in sys.path:
        sys.path.append(path)
import pytorch_lightning as pl
import os
from pytorch_lightning.utilities.parsing import AttributeDict
from pytorch_lightning.loggers import TensorBoardLogger
import numpy as np
import matplotlib.pyplot as plt
import torch
import cv2
import albumentations
import torchvision.transforms as transforms
import torchvision

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
from sparse_feature_pyramid.model import SparseFeaturePyramidAutoencoder
from sparse_feature_pyramid.data import SevenScenesDataModule
from sparse_feature_pyramid.utils import UniversalFactory
from sparse_feature_pyramid.utils.clearml_figure_reporter import ClearmlFigureReporter

factory = UniversalFactory([SparseFeaturePyramidAutoencoder])

In [4]:
from clearml import Task, Logger

# Load model

In [5]:
# Тренируем на всех сценах
image_size = 256
data_module_parameters = {
    "batch_size": 16,
    "num_workers": 4,
    "image_size": image_size,
    "scenes": ["fire"], #, "chess", "pumpkin", "stairs", "heads", "office", "redkitchen"],
    "center_crop": True,
    "random_jitter": True,
    "random_rotation": True,
    "root_dataset_path": "/home/andrei/media/7scenes"
}

scene = data_module_parameters["scenes"][0]
data_module = SevenScenesDataModule(**data_module_parameters)
model_parameters = AttributeDict(
    name="SparseFeaturePyramidAutoencoder",
    optimizer=AttributeDict(),
    feature_dimensions=[8, 16, 32, 64, 128],
    size_loss_koef=(image_size*image_size*3) * (1 / 500000.),
    input_dimension=3,
    kl_loss_coefficient=1
)
model = factory.make_from_parameters(model_parameters)
model.set_figure_reporter(ClearmlFigureReporter())

[ToyDataModule] - train subset size 2000
[ToyDataModule] - validation dataset size 2000


In [6]:
checkpoint = torch.load("/home/andrei/git/sparse-feature-pyramid/notebook/version_14/checkpoints/epoch=89-step=36629.ckpt")
model.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [7]:
def get_image(batch, index):
    input_image = batch["image"][index].detach().cpu().numpy().transpose(1, 2, 0)
    input_image = input_image * np.array(data_module._std)[None, None] + np.array(data_module._mean)[None, None]
    input_image = np.clip(input_image, 0, 1)
    return input_image

# Extract points

In [8]:
from kapture_localization.matching.matching import MatchPairNnTorch
from torch.nn.functional import grid_sample
import argparse
import numpy as np
import imageio
import torch
from tqdm import tqdm
from PIL import Image
import scipy
import scipy.io
import scipy.misc
import kornia
import cv2

In [9]:
def get_keypoint_grid(output, level):
    descriptors = output[1][level]
    height = descriptors.shape[2]
    width = descriptors.shape[3]
    batch_size = descriptors.shape[0]
    x, y = np.meshgrid(range(height), range(width))
    keypoints = np.array([x, y]).transpose(1, 2, 0).astype(np.float32)
    keypoints[:, :, 0] = 2 * keypoints[:, :, 0] / (width - 1) - 1
    keypoints[:, :, 1] = 2 * keypoints[:, :, 1] / (height - 1) - 1
    keypoints = torch.tensor(keypoints)[None]
    return torch.repeat_interleave(keypoints, batch_size, dim=0)
    
def get_numpy_descriptors(descriptors, index):
    descriptors = descriptors[index].detach().cpu().numpy().transpose(1, 2, 0)
    feature_dimension = descriptors.shape[-1]
    descriptors = descriptors.reshape(-1, feature_dimension)
    descriptors = descriptors / (np.linalg.norm(descriptors, axis=1)[:, None] + 1e-9)
    return descriptors

def get_interpolated_descriptors(output, index, level, keypoint_grid):
    descriptors = grid_sample(output[1][level], keypoint_grid, align_corners=True)
    return get_numpy_descriptors(descriptors, index)


def get_updated_descriptors(output, index, level):
    masked_feature_pyramid = [feature * mask for feature, mask in zip(output[1], output[2])]
    x = masked_feature_pyramid[-1]
    for i in range(3 - level):
        x = model._decoder_blocks[i](x, masked_feature_pyramid[-i - 2])

    x = model._decoder_blocks[3 - level]._upsample_conv(x)
    x = torch.cat([x, masked_feature_pyramid[level]], dim=1)
    return get_numpy_descriptors(x, index)

def get_mask(output, index, level):
    mask = output[2][level][index][0].detach().cpu().numpy().reshape(-1)
    return mask.astype(bool)

def get_descriptors(output, index, level):
    return get_numpy_descriptors(output[1][level], index)

def get_keypoints(output, index, level):
    descriptors = output[1][level][index].detach().cpu().numpy().transpose(1, 2, 0)
    height = descriptors.shape[0]
    width = descriptors.shape[1]
    x, y = np.meshgrid(range(height), range(width))
    keypoints = np.array([x, y]).transpose(1, 2, 0).reshape(-1, 2)
    keypoints = keypoints / (width - 1) * (width * 2 ** level - 1)
    return keypoints

def get_descriptors_and_keypoints(output, index, level):
    keypoints = get_keypoints(output, index, level)
    descriptors = get_descriptors(output, index, level)
    mask = get_mask(output, index, level)
    return keypoints[mask], descriptors[mask]

def get_interpolated_descriptors_and_keypoints(output, index, level, descriptor_levels):
    keypoints = get_keypoints(output, index, level)
    keypoint_grid = get_keypoint_grid(output, level)
    descriptors = []
    for descriptor_level in descriptor_levels:
        descriptors.append(get_interpolated_descriptors(output, index, descriptor_level, keypoint_grid))
    descriptors = np.concatenate(descriptors, axis=1)
    mask = get_mask(output, index, level)
    return keypoints[mask], descriptors[mask]

def generate_read_function(method, extension='ppm'):
    def read_function(seq_name, im_idx):
        aux = np.load(os.path.join(dataset_path, seq_name, '%d.%s.%s' % (im_idx, extension, method)), allow_pickle=True)
        if top_k is None:
            return aux['keypoints'], aux['descriptors']
        else:
            assert('scores' in aux)
            ids = np.argsort(aux['scores'])[-top_k :]
            return aux['keypoints'][ids, :], aux['descriptors'][ids, :]
    return read_function

/home/andrei/miniconda3/envs/enGAN/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
convert_tensor = transforms.ToTensor()

keypoints = []
scores = []
descriptors = []

image_list_file = '/home/andrei/git/d2-net/image_list_hpatches_sequences.txt'
newsize = (256, 256)
level = 2
descript = [2, 3, 4]

with open(image_list_file, 'r') as f:
    lines = f.readlines()
    
for line in tqdm(lines, total=len(lines)):
    path = "/home/andrei/git/d2-net/" + line.strip()

    # reshape image
    im = Image.open(path)
    w, h = im.size
    im = im.resize(newsize)
    image_orig = np.array(im)
    
    # normalize image
    image = image_orig/256
    
    # cinvert to proper shape for NN
    image = (image - np.array(data_module._mean))/np.array(data_module._std)
    image_batch = convert_tensor(image.astype('float32'))[None]
    output = model(image_batch)

    # obtain kp and descriptors. No scores for now
    keypoints, descriptors = get_interpolated_descriptors_and_keypoints(output, 0, level, descript)
    scores = np.zeros((keypoints.shape[0], 1))
    
    # return back to original size
    keypoints[:, 0] = keypoints[:, 0] * (w/256)
    keypoints[:, 1] = keypoints[:, 1] * (h/256)
    
    # save each image as npy file
    with open(path + '.SFP', 'wb') as output_file:
        np.savez(
            output_file,
            keypoints=keypoints,
            scores=scores,
            descriptors=descriptors
        )

/home/andrei/miniconda3/envs/enGAN/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 648/648 [01:06<00:00,  9.75it/s]
